In [10]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIR = Path(".")
SRC_XLS = DATA_DIR / "CTG.xls"
OUT_CSV = DATA_DIR / "CTG_clean.csv"
OUT_XLSX = DATA_DIR / "CTG_clean.xlsx"  # optional

xls = pd.ExcelFile("CTG.xls")
print(xls.sheet_names)

['Description', 'Data', 'Raw Data']


In [16]:
df = pd.read_excel("CTG.xls", sheet_name="Raw Data")
df.columns = df.columns.str.strip()
df = df.dropna(how="all")
df.head()

,FileName,Date,SegFile,b,e,LBE,LB,AC,FM,UC,...,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
1,Variab10.txt,1996-12-01,CTG0001.txt,240.0,357.0,120.0,120.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,2.0
2,Fmcs_1.txt,1996-05-03,CTG0002.txt,5.0,632.0,132.0,132.0,4.0,0.0,4.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
3,Fmcs_1.txt,1996-05-03,CTG0003.txt,177.0,779.0,133.0,133.0,2.0,0.0,5.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
4,Fmcs_1.txt,1996-05-03,CTG0004.txt,411.0,1192.0,134.0,134.0,2.0,0.0,6.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
5,Fmcs_1.txt,1996-05-03,CTG0005.txt,533.0,1147.0,132.0,132.0,4.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0


In [35]:
def clean_col(c):
    if pd.isna(c):
        return "unnamed"
    return (
        str(c)
        .strip()
        .replace("/", "_")
        .replace("\\", "_")
        .replace("-", "_")
        .replace("(", "")
        .replace(")", "")
        .replace("%", "pct")
        .replace(".", "_")
        .replace(" ", "_")
        .lower()
    )

df.columns = [clean_col(c) for c in df.columns]
df = df.loc[:, ~df.columns.duplicated()]


df.head(3)
print(df.columns.tolist())

['filename', 'date', 'segfile', 'b', 'e', 'lbe', 'lb', 'ac', 'fm', 'uc', 'astv', 'mstv', 'altv', 'mltv', 'dl', 'ds', 'dp', 'dr', 'width', 'min', 'max', 'nmax', 'nzeros', 'mode', 'mean', 'median', 'variance', 'tendency', 'a', 'c', 'd', 'ad', 'de', 'ld', 'fs', 'susp', 'class', 'nsp']


In [36]:
target_candidates = [c for c in df.columns if "nsp" in c.lower()]
target_col_std = target_candidates[0] if target_candidates else None

feature_cols = [c for c in df.columns if c != target_col_std]

for c in feature_cols:
    if df[c].dtype == "object":
        df[c] = pd.to_numeric(df[c], errors="coerce")

df.dtypes.head(10)

filename    float64
date          int64
segfile     float64
b           float64
e           float64
lbe         float64
lb          float64
ac          float64
fm          float64
uc          float64
dtype: object

In [37]:
print("Shape:", df.shape)
print("Missing values per column (top 10):")
print(df.isna().sum().sort_values(ascending=False).head(10))

dup_count = df.duplicated().sum()
print("Duplicate rows:", dup_count)

Shape: (2118, 38)
Missing values per column (top 10):
filename    2118
segfile     2118
e              2
b              2
lbe            2
lb             2
fs             2
ac             2
max            2
nmax           2
dtype: int64
Duplicate rows: 0


In [38]:
before = len(df)
df = df.drop_duplicates()
after = len(df)
print(f"Duplicates removed: {before - after} (now {after} rows)")

Duplicates removed: 0 (now 2118 rows)


In [39]:
df_clean = df.copy()
for c in feature_cols:
    if df_clean[c].isna().any():
        df_clean[c] = df_clean[c].fillna(df_clean[c].median())

# Rename target back to exact 'NSP' for downstream consistency
df_clean = df_clean.rename(columns={target_col_std: "NSP"})

# Sanity: missing after impute (should be 0 except the target if there were blanks)
df_clean.isna().sum().sort_values(ascending=False).head(10)


C:\Users\bang kai\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\bang kai\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


filename    2118
segfile     2118
NSP            2
date           0
e              0
b              0
lbe            0
lb             0
fm             0
ac             0
dtype: int64

In [40]:
df_clean.to_csv(OUT_CSV, index=False)
with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter") as writer:
    df_clean.to_excel(writer, index=False, sheet_name="cleaned")

print("Saved CSV ->", OUT_CSV.resolve())
print("Saved Excel ->", OUT_XLSX.resolve())
df_clean.head(5)

Saved CSV -> C:\Users\bang kai\Desktop\Datathon\CTG_clean.csv
Saved Excel -> C:\Users\bang kai\Desktop\Datathon\CTG_clean.xlsx


,filename,date,segfile,b,e,lbe,lb,ac,fm,uc,...,a,c,d,ad,de,ld,fs,susp,class,NSP
1,NaN,849398400000000000,NaN,240.0,357.0,120.0,120.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,2.0
2,NaN,831081600000000000,NaN,5.0,632.0,132.0,132.0,4.0,0.0,4.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
3,NaN,831081600000000000,NaN,177.0,779.0,133.0,133.0,2.0,0.0,5.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
4,NaN,831081600000000000,NaN,411.0,1192.0,134.0,134.0,2.0,0.0,6.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
5,NaN,831081600000000000,NaN,533.0,1147.0,132.0,132.0,4.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
